Tip: I used Google Colab and I mounted my drive (see below)
To run it, please ensure you import the same structure as in the 2nd cell, and not only the notebook

I thought it would be interesting to see what could be done just from the article. The aim was to see if I could, from Hugging Face's pipeline, and the information in the article, reconstruct their work (their code).

In [19]:
# pipeline from Hugging Face
import torch
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image
import numpy as np

model_id = "mlpc-lab/TokenCompose_SD14_A"

pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float32)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [20]:
import sys
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab/Projet
!pwd  # check that you're in the right directory
!ls # you should see coco_gsam_seg and src.ipynb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab/Projet
/content/drive/MyDrive/Colab/Projet
coco_gsam_seg  src.ipynb


### CPU (plantage RAM assez rapide)

In [21]:
'''
data = [
    {
        "file_name": "000000000142.jpg",
        "text": "Plate with a piece of bread, dark chocolate spread, bananas and a carton of Silk.",
        "attn_list": [
            ["piece", "seg/000000000142/mask_000000000142_piece.png"],
            ["carton", "seg/000000000142/mask_000000000142_carton.png"],
            ["Plate", "seg/000000000142/mask_000000000142_Plate.png"],
            ["Silk", "seg/000000000142/mask_000000000142_Silk.png"],
            ["bananas", "seg/000000000142/mask_000000000142_bananas.png"],
            ["bread", "seg/000000000142/mask_000000000142_bread.png"],
            ["chocolate spread", "seg/000000000142/mask_000000000142_chocolate_spread.png"]
        ]
    },
    {
        "file_name": "000000000370.jpg",
        "text": "A little girl holds a piece of broccoli towards the camera.",
        "attn_list": [
            ["girl", "seg/000000000370/mask_000000000370_girl.png"],
            ["broccoli", "seg/000000000370/mask_000000000370_broccoli.png"]
        ]
    }
]

tok=pipe.components['tokenizer']
enc=pipe.components['text_encoder']
vae=pipe.components['vae']
une=pipe.components['unet']

for d in data:
  # récupération du prompt et passage par le tokenizer puis text_encoder
  toks=tok(text=d["text"],return_tensors='pt')
  encs=enc(input_ids=toks['input_ids'],attention_mask=toks['attention_mask'])
  print(encs.last_hidden_state.shape)

  # récupération de l'image et passage dans le vae
  photo_path="./coco_gsam_seg/"+d["file_name"][0:-4]+"/src.jpg"
  image=Image.open(photo_path)
  image=image.resize((512, 512))
  image_array=np.array(image)
  image_tensor=torch.tensor(image_array, dtype=torch.float32).permute(2,0,1)/255.0
  print(image_tensor.shape)
  lat=vae.encode(image_tensor.unsqueeze(0))
  lat_tens=lat.latent_dist.sample()
  print(lat_tens.shape)

'''

'\ndata = [\n    {\n        "file_name": "000000000142.jpg",\n        "text": "Plate with a piece of bread, dark chocolate spread, bananas and a carton of Silk.",\n        "attn_list": [\n            ["piece", "seg/000000000142/mask_000000000142_piece.png"],\n            ["carton", "seg/000000000142/mask_000000000142_carton.png"],\n            ["Plate", "seg/000000000142/mask_000000000142_Plate.png"],\n            ["Silk", "seg/000000000142/mask_000000000142_Silk.png"],\n            ["bananas", "seg/000000000142/mask_000000000142_bananas.png"],\n            ["bread", "seg/000000000142/mask_000000000142_bread.png"],\n            ["chocolate spread", "seg/000000000142/mask_000000000142_chocolate_spread.png"]\n        ]\n    },\n    {\n        "file_name": "000000000370.jpg",\n        "text": "A little girl holds a piece of broccoli towards the camera.",\n        "attn_list": [\n            ["girl", "seg/000000000370/mask_000000000370_girl.png"],\n            ["broccoli", "seg/00000000037

### GPU (small test with 2 dataset images and batch size=1)

In [22]:
if torch.cuda.is_available():
  device=torch.device("cuda")
  print(f"using GPU")
else:
  device=torch.device("cpu")
  print(f"using CPU")

# for the moment we are only considering the first two lines of the dataset
data = [
    {
        "file_name": "000000000142.jpg",
        "text": "Plate with a piece of bread, dark chocolate spread, bananas and a carton of Silk.",
        "attn_list": [
            ["piece", "seg/000000000142/mask_000000000142_piece.png"],
            ["carton", "seg/000000000142/mask_000000000142_carton.png"],
            ["Plate", "seg/000000000142/mask_000000000142_Plate.png"],
            ["Silk", "seg/000000000142/mask_000000000142_Silk.png"],
            ["bananas", "seg/000000000142/mask_000000000142_bananas.png"],
            ["bread", "seg/000000000142/mask_000000000142_bread.png"],
            ["chocolate spread", "seg/000000000142/mask_000000000142_chocolate_spread.png"]
        ]
    },
    {
        "file_name": "000000000370.jpg",
        "text": "A little girl holds a piece of broccoli towards the camera.",
        "attn_list": [
            ["girl", "seg/000000000370/mask_000000000370_girl.png"],
            ["broccoli", "seg/000000000370/mask_000000000370_broccoli.png"]
        ]
    }
]

# load components
tok=pipe.components['tokenizer']
enc=pipe.components['text_encoder'].to(device)
vae=pipe.components['vae'].to(device)
une=pipe.components['unet'].to(device)
sch=pipe.components['scheduler']

for d in data:
    # Retrieve the prompt and run it through the tokenizer (CPU) then text_encoder (GPU)
    toks=tok(text=d["text"],return_tensors='pt') # if I don't fill in return_tensors, the elements of toks will be lists (but encs takes tensors as inputs) -> 'pt'="pytorch"
    toks={Key:Value.to(device) for Key,Value in toks.items()}  # transfer the data on GPU
    encs=enc(input_ids=toks['input_ids'],attention_mask=toks['attention_mask'])
    print(encs.last_hidden_state.shape)
    # torch.Size([1, 20, 768]) for the 1st image
    # torch.Size([1, 14, 768]) for the 2nd image (only the length of sequence varies)
    # (batch size, sequence length, dimension of text embedding)

    # Retrieving the image and transferring it to the vae
    photo_path = "./coco_gsam_seg/" + d["file_name"][0:-4] + "/src.jpg"
    photo=Image.open(photo_path)
    photo=photo.resize((512, 512))
    photo_array=np.array(photo)
    photo_tensor=torch.tensor(photo_array, dtype=torch.float32).permute(2,0,1)/255.0
    photo_tensor=photo_tensor.unsqueeze(0).to(device)  # vae takes 4-dimension tensor as input so we must add a first dimension artificially
    print(photo_tensor.shape)
    # torch.Size([1, 3, 512, 512])

    # encode the photo with autoencoder kl to obtain latent
    lat=vae.encode(photo_tensor)
    lat_tens=lat.latent_dist.sample() # obtain the tensor associated with the latent
    print(lat_tens.shape)
    # torch.Size([1, 4, 64, 64])

    # add noise
    # here I used the code from https://github.com/mlpc-ucsd/TokenCompose
    noise = torch.randn_like(lat_tens)
    bsz = lat_tens.shape[0]
    max_timestep = sch.config.num_train_timesteps
    timesteps = torch.randint(0, max_timestep, (bsz,), device=lat_tens.device)
    timesteps = timesteps.long()
    noisy_latents = sch.add_noise(lat_tens, noise, timesteps)
    print(noisy_latents.shape)
    # torch.Size([1, 4, 64, 64])

    # use unet
    prediction=une(noisy_latents,timesteps,encs.last_hidden_state).sample
    print(prediction.shape)
    # torch.Size([1, 4, 64, 64])

    # Loss LDM
    loss_ldm=torch.nn.functional.mse_loss(prediction,lat_tens)
    print(f"loss_ldm: {loss_ldm}")
    # tensor(16.7185, device='cuda:0', grad_fn=<MseLossBackward0>) for the 1st image
    # tensor(19.2932, device='cuda:0', grad_fn=<MseLossBackward0>) fir the 2nd image

    # Ltoken
    ltoken=0
    for token in d["attn_list"]:
      photo_path_token="./coco_gsam_"+token[1] # obtain the path to the attention mask
      photo_token=Image.open(photo_path_token)
      photo_token=photo_token.resize((64, 64)) # convert to the size of the latent
      photo_array_token=np.array(photo_token)
      photo_tensor_token=torch.tensor(photo_array_token, dtype=torch.float32).permute(2,0,1)/255.0
      print(photo_tensor_token.shape)
      # torch.Size([4, 64, 64])

      photo_tensor_token=photo_tensor_token.to(device)
      photo_tensor_token=photo_tensor_token.unsqueeze(0)  # add a dimension to go from [4, 64, 64] to [1, 4, 64, 64]
      pixelwise_product=photo_tensor_token*prediction  # normally we use the attention activations but for the moment I don't have them (we look at that later and use "prediction" instead for the moment)
      sum_pixelwise=pixelwise_product.sum()
      sum_prediction=prediction.sum()
      ratio=sum_pixelwise/sum_prediction
      result=(1-ratio)**2
      ltoken+=result
    print(f"ltoken: {ltoken}")

    # Lpixel
    # disabled because not working properly
    '''
    lpixel=0
    for token in d["attn_list"]:
      photo_path_token="./coco_gsam_"+token[1] # obtain the path to the attention mask
      photo_token=Image.open(photo_path_token)
      photo_token=photo_token.resize((64, 64)) # convert to the size of the latent
      photo_array_token=np.array(photo_token)
      photo_tensor_token=torch.tensor(photo_array_token, dtype=torch.float32).permute(2,0,1)/255.0
      print(photo_tensor_token.shape)
      # torch.Size([4, 64, 64])

      photo_tensor_token=photo_tensor_token.to(device)
      photo_tensor_token=photo_tensor_token.unsqueeze(0)  # add a dimension to go from [4, 64, 64] to [1, 4, 64, 64]
      log_prediction=torch.log(prediction)  # log(prediction)
      log_one_minus_prediction=torch.log(1-prediction)  # log(1-prediction)
      a=image_tensor_token*log_prediction  # image_tensor_token[u]*log(prediction[u])
      b=(1-image_tensor_token)*log_one_minus_prediction  # (1-image_tensor_token[u])*log(1-prediction[u])
      final_result=(a+b).sum()
      lpixel+=final_result
    print(f"lpixel: {lpixel}")
    '''

    loss=loss_ldm+ltoken
    print(loss)


using GPU
torch.Size([1, 20, 768])
torch.Size([1, 3, 512, 512])
torch.Size([1, 4, 64, 64])
torch.Size([1, 4, 64, 64])
torch.Size([1, 4, 64, 64])
loss_ldm: 16.387744903564453
torch.Size([4, 64, 64])
torch.Size([4, 64, 64])
torch.Size([4, 64, 64])
torch.Size([4, 64, 64])
torch.Size([4, 64, 64])
torch.Size([4, 64, 64])
torch.Size([4, 64, 64])
ltoken: 9.494438171386719
tensor(25.8822, device='cuda:0', grad_fn=<AddBackward0>)
torch.Size([1, 14, 768])
torch.Size([1, 3, 512, 512])
torch.Size([1, 4, 64, 64])
torch.Size([1, 4, 64, 64])
torch.Size([1, 4, 64, 64])
loss_ldm: 23.554521560668945
torch.Size([4, 64, 64])
torch.Size([4, 64, 64])
ltoken: 1.4836835861206055
tensor(25.0382, device='cuda:0', grad_fn=<AddBackward0>)


### GPU (small test with 2 dataset images and batch size=2)
Big difference with before: remember to add `padding=True` in the tokenizer

In [23]:
if torch.cuda.is_available():
  device=torch.device("cuda")
  print(f"using GPU")
else:
  device=torch.device("cpu")
  print(f"using CPU")

data = [
    {
        "file_name": "000000000142.jpg",
        "text": "Plate with a piece of bread, dark chocolate spread, bananas and a carton of Silk.",
        "attn_list": [
            ["piece", "seg/000000000142/mask_000000000142_piece.png"],
            ["carton", "seg/000000000142/mask_000000000142_carton.png"],
            ["Plate", "seg/000000000142/mask_000000000142_Plate.png"],
            ["Silk", "seg/000000000142/mask_000000000142_Silk.png"],
            ["bananas", "seg/000000000142/mask_000000000142_bananas.png"],
            ["bread", "seg/000000000142/mask_000000000142_bread.png"],
            ["chocolate spread", "seg/000000000142/mask_000000000142_chocolate_spread.png"]
        ]
    },
    {
        "file_name": "000000000370.jpg",
        "text": "A little girl holds a piece of broccoli towards the camera.",
        "attn_list": [
            ["girl", "seg/000000000370/mask_000000000370_girl.png"],
            ["broccoli", "seg/000000000370/mask_000000000370_broccoli.png"]
        ]
    }
]

# load components
tok=pipe.components['tokenizer']
enc=pipe.components['text_encoder'].to(device)
vae=pipe.components['vae'].to(device)
une=pipe.components['unet'].to(device)
sch=pipe.components['scheduler']

# create lists
photos=[]
texts=[]
attn_lists=[]

# collect data
for d in data:
    texts.append(d["text"])
    attn_lists.append(d["attn_list"])
    photo_path="./coco_gsam_seg/" + d["file_name"][0:-4] + "/src.jpg"
    photo=Image.open(photo_path)
    photo=photo.resize((512, 512))
    photo_array=np.array(photo)
    photo_tensor=torch.tensor(photo_array, dtype=torch.float32).permute(2,0,1)/255.0
    photos.append(photo_tensor)

# convert lists to tensors
texts_batch=tok(text=texts, return_tensors='pt',padding=True) # wde must activate padding here
texts_batch={Key: Value.to(device) for Key,Value in texts_batch.items()}
photos_batch=torch.stack([photo.to(device) for photo in photos])

# go through encoder
encs=enc(input_ids=texts_batch['input_ids'], attention_mask=texts_batch['attention_mask'])

# go through vae
latents=vae.encode(photos_batch)
lat_tens=latents.latent_dist.sample() # recover specifically the tensor

# add noise
# here I used the code from https://github.com/mlpc-ucsd/TokenCompose
noise = torch.randn_like(lat_tens)
bsz = lat_tens.shape[0]  # Taille du batch
max_timestep=sch.config.num_train_timesteps
timesteps = torch.randint(0, max_timestep, (bsz,), device=lat_tens.device)
timesteps = timesteps.long()
noisy_latents=sch.add_noise(lat_tens, noise, timesteps)

# use unet
predictions=une(noisy_latents,timesteps,encs.last_hidden_state).sample

# Loss LDM
loss_ldm=torch.nn.functional.mse_loss(predictions,lat_tens)

# Ltoken and Lpixel
ltoken=0.0
for k, d in enumerate(data):
    for token in d["attn_list"]:
        photo_path_token="./coco_gsam_"+token[1]
        photo_token=Image.open(photo_path_token)
        photo_token=photo_token.resize((64, 64)) # convert to the size of the latent
        photo_array_token=np.array(photo_token)
        photo_tensor_token=torch.tensor(photo_array_token, dtype=torch.float32).permute(2,0,1)/255.0

        photo_tensor_token=photo_tensor_token.to(device)
        photo_tensor_token=photo_tensor_token.unsqueeze(0)

        # pixekwise product
        pixelwise_product=photo_tensor_token*predictions[k:k+1]  # instead of using the cross attention map I use predictions for the moment because I don't know how to get the cross attention map
        sum_pixelwise=pixelwise_product.sum()
        sum_prediction=predictions[k:k+1].sum()
        ratio=sum_pixelwise/sum_prediction
        result=(1-ratio)**2
        ltoken+=result

# total loss
loss=loss_ldm +ltoken
print(loss)


using GPU
tensor(31.3645, device='cuda:0', grad_fn=<AddBackward0>)


### Where is the cross attention map ?

In [24]:
une.attn_processors

{'down_blocks.0.attentions.0.transformer_blocks.0.attn1.processor': <diffusers.models.attention_processor.AttnProcessor2_0 at 0x7995e8a54a90>,
 'down_blocks.0.attentions.0.transformer_blocks.0.attn2.processor': <diffusers.models.attention_processor.AttnProcessor2_0 at 0x7995e8a56e60>,
 'down_blocks.0.attentions.1.transformer_blocks.0.attn1.processor': <diffusers.models.attention_processor.AttnProcessor2_0 at 0x7995e8a544f0>,
 'down_blocks.0.attentions.1.transformer_blocks.0.attn2.processor': <diffusers.models.attention_processor.AttnProcessor2_0 at 0x7995e8a56e30>,
 'down_blocks.1.attentions.0.transformer_blocks.0.attn1.processor': <diffusers.models.attention_processor.AttnProcessor2_0 at 0x7995e89110c0>,
 'down_blocks.1.attentions.0.transformer_blocks.0.attn2.processor': <diffusers.models.attention_processor.AttnProcessor2_0 at 0x7995e8912800>,
 'down_blocks.1.attentions.1.transformer_blocks.0.attn1.processor': <diffusers.models.attention_processor.AttnProcessor2_0 at 0x7995e8911cc0>,

### Try recovering attention map with a hook

Using the previous code GPU (small test with 2 dataset images and batch size=1)

In [25]:
if torch.cuda.is_available():
  device=torch.device("cuda")
  print(f"using GPU")
else:
  device=torch.device("cpu")
  print(f"using CPU")

# for the moment we are only considering the first two lines of the dataset
data = [
    {
        "file_name": "000000000142.jpg",
        "text": "Plate with a piece of bread, dark chocolate spread, bananas and a carton of Silk.",
        "attn_list": [
            ["piece", "seg/000000000142/mask_000000000142_piece.png"],
            ["carton", "seg/000000000142/mask_000000000142_carton.png"],
            ["Plate", "seg/000000000142/mask_000000000142_Plate.png"],
            ["Silk", "seg/000000000142/mask_000000000142_Silk.png"],
            ["bananas", "seg/000000000142/mask_000000000142_bananas.png"],
            ["bread", "seg/000000000142/mask_000000000142_bread.png"],
            ["chocolate spread", "seg/000000000142/mask_000000000142_chocolate_spread.png"]
        ]
    },
    {
        "file_name": "000000000370.jpg",
        "text": "A little girl holds a piece of broccoli towards the camera.",
        "attn_list": [
            ["girl", "seg/000000000370/mask_000000000370_girl.png"],
            ["broccoli", "seg/000000000370/mask_000000000370_broccoli.png"]
        ]
    }
]

# load components
tok=pipe.components['tokenizer']
enc=pipe.components['text_encoder'].to(device)
vae=pipe.components['vae'].to(device)
une=pipe.components['unet'].to(device)
sch=pipe.components['scheduler']

for d in data:
    # Retrieve the prompt and run it through the tokenizer (CPU) then text_encoder (GPU)
    toks=tok(text=d["text"],return_tensors='pt') # if I don't fill in return_tensors, the elements of toks will be lists (but encs takes tensors as inputs) -> 'pt'="pytorch"
    toks={Key:Value.to(device) for Key,Value in toks.items()}  # transfer the data on GPU
    encs=enc(input_ids=toks['input_ids'],attention_mask=toks['attention_mask'])
    print(encs.last_hidden_state.shape)
    # torch.Size([1, 20, 768]) for the 1st image
    # torch.Size([1, 14, 768]) for the 2nd image (only the length of sequence varies)
    # (batch size, sequence length, dimension of text embedding)

    # Retrieving the image and transferring it to the vae
    photo_path = "./coco_gsam_seg/" + d["file_name"][0:-4] + "/src.jpg"
    photo=Image.open(photo_path)
    photo=photo.resize((512, 512))
    photo_array=np.array(photo)
    photo_tensor=torch.tensor(photo_array, dtype=torch.float32).permute(2,0,1)/255.0
    photo_tensor=photo_tensor.unsqueeze(0).to(device)  # vae takes 4-dimension tensor as input so we must add a first dimension artificially
    print(photo_tensor.shape)
    # torch.Size([1, 3, 512, 512])

    # encode the photo with autoencoder kl to obtain latent
    lat=vae.encode(photo_tensor)
    lat_tens=lat.latent_dist.sample() # obtain the tensor associated with the latent
    print(lat_tens.shape)
    # torch.Size([1, 4, 64, 64])

    # add noise
    # here I used the code from https://github.com/mlpc-ucsd/TokenCompose
    noise = torch.randn_like(lat_tens)
    bsz = lat_tens.shape[0]
    max_timestep = sch.config.num_train_timesteps
    timesteps = torch.randint(0, max_timestep, (bsz,), device=lat_tens.device)
    timesteps = timesteps.long()
    noisy_latents = sch.add_noise(lat_tens, noise, timesteps)
    print(noisy_latents.shape)
    # torch.Size([1, 4, 64, 64])

    '''
    We add the hook just right here
    '''
    # prepare list containing activations
    activations=[]

    # I add a function that I use with the hook
    def attention_activations(module, input, output):
          activations.append(output)

    transformer_block=une.down_blocks[0].attentions[0].transformer_blocks[0]  # the block from which I want to get activations
    transformer_block.register_forward_hook(attention_activations)

    # use unet
    prediction=une(noisy_latents,timesteps,encs.last_hidden_state).sample
    print(prediction.shape)
    # torch.Size([1, 4, 64, 64])

    # Loss LDM
    loss_ldm=torch.nn.functional.mse_loss(prediction,lat_tens)
    print(f"loss_ldm: {loss_ldm}")
    # tensor(16.7185, device='cuda:0', grad_fn=<MseLossBackward0>) for the 1st image
    # tensor(19.2932, device='cuda:0', grad_fn=<MseLossBackward0>) fir the 2nd image

    # Ltoken
    ltoken=0
    for token in d["attn_list"]:
      photo_path_token="./coco_gsam_"+token[1] # obtain the path to the attention mask
      photo_token=Image.open(photo_path_token)
      photo_token=photo_token.resize((64, 64)) # convert to the size of the latent
      photo_array_token=np.array(photo_token)
      photo_tensor_token=torch.tensor(photo_array_token, dtype=torch.float32).permute(2,0,1)/255.0
      print(photo_tensor_token.shape)
      # torch.Size([4, 64, 64])

      photo_tensor_token=photo_tensor_token.to(device)
      photo_tensor_token=photo_tensor_token.unsqueeze(0)  # add a dimension to go from [4, 64, 64] to [1, 4, 64, 64]
      pixelwise_product=photo_tensor_token*prediction  # normally we use the attention activations but for the moment I don't have them (we look at that later and use "prediction" instead for the moment)
      sum_pixelwise=pixelwise_product.sum()
      sum_prediction=prediction.sum()
      ratio=sum_pixelwise/sum_prediction
      result=(1-ratio)**2
      ltoken+=result
    print(f"ltoken: {ltoken}")

    # Lpixel
    # disabled because not working properly
    '''
    lpixel=0
    for token in d["attn_list"]:
      photo_path_token="./coco_gsam_"+token[1] # obtain the path to the attention mask
      photo_token=Image.open(photo_path_token)
      photo_token=photo_token.resize((64, 64)) # convert to the size of the latent
      photo_array_token=np.array(photo_token)
      photo_tensor_token=torch.tensor(photo_array_token, dtype=torch.float32).permute(2,0,1)/255.0
      print(photo_tensor_token.shape)
      # torch.Size([4, 64, 64])

      photo_tensor_token=photo_tensor_token.to(device)
      photo_tensor_token=photo_tensor_token.unsqueeze(0)  # add a dimension to go from [4, 64, 64] to [1, 4, 64, 64]
      log_prediction=torch.log(prediction)  # log(prediction)
      log_one_minus_prediction=torch.log(1-prediction)  # log(1-prediction)
      a=image_tensor_token*log_prediction  # image_tensor_token[u]*log(prediction[u])
      b=(1-image_tensor_token)*log_one_minus_prediction  # (1-image_tensor_token[u])*log(1-prediction[u])
      final_result=(a+b).sum()
      lpixel+=final_result
    print(f"lpixel: {lpixel}")
    '''

    loss=loss_ldm+ltoken
    print(loss)

using GPU
torch.Size([1, 20, 768])
torch.Size([1, 3, 512, 512])
torch.Size([1, 4, 64, 64])
torch.Size([1, 4, 64, 64])
torch.Size([1, 4, 64, 64])
loss_ldm: 16.33377456665039
torch.Size([4, 64, 64])
torch.Size([4, 64, 64])
torch.Size([4, 64, 64])
torch.Size([4, 64, 64])
torch.Size([4, 64, 64])
torch.Size([4, 64, 64])
torch.Size([4, 64, 64])
ltoken: 10.115011215209961
tensor(26.4488, device='cuda:0', grad_fn=<AddBackward0>)
torch.Size([1, 14, 768])
torch.Size([1, 3, 512, 512])
torch.Size([1, 4, 64, 64])
torch.Size([1, 4, 64, 64])
torch.Size([1, 4, 64, 64])
loss_ldm: 21.954076766967773
torch.Size([4, 64, 64])
torch.Size([4, 64, 64])
ltoken: 1.3780158758163452
tensor(23.3321, device='cuda:0', grad_fn=<AddBackward0>)


In [26]:
print(activations[1].shape) # renvoie [1,4096,320]. Or 4096 correspond à 64x64, clairement le nombre de cases de lat_tens
# Mais à quoi correspond 320 ?

torch.Size([1, 4096, 320])


Ce que j'ai fait jusque là (fr, see english below):
* étudier en détail le code sur Hugging Face de S.D.1.4
* lire le papier (-> il y a une erreur sur le schéma mais globalement bon papier)
* tenter d'exécuter TokenCompose (-> j'ai eu un problème de bibliothèque)
* recoder mon propre TokenCompose grâce à ma connaissance du papier et des modules d'Hugging Face. Je me suis dit que ce serait finalement le meilleur moyen de comprendre ce qui se passe dans TokenCompose et unet. Je n'ai pas bien compris comment récupérer la carte des activations (dans les attentions layers) car dimension [1,4096,320] (j'ai bien compris que 4096 c'est pour 64x64 car l'espace latent est 64x64 mais je n'ai pas compris le rapport entre 320 et les dimensions de l'embedding textuel.

Mes enseignements sur TokenCompose:
* papier bien rédigé sauf petite erreur sur la figure 3 (les flèches sont mal reliées)
* j'ai pris conscience que la vraie difficulté à laquelle ils ont dû faire face est de récupérer les attentions (car je n'ai pas réussi pour l'instant)... mais j'ai bien compris que théoriquement avec un `hook` ça doit être possible

Enseignements généraux:
* j'ai appris à manipuler Hugging Face (la plateforme)
* J'ai réalisé qu'Hugging Face est faite pour que les modèles soient utilisés, mais pas forcément fine tunés. Par exemple, SD14 n'a pas de fonction forward. il n'a qu'une fonction __call__
* on peut facilement accéder au code des modèles sur Hugging Face mais difficile à lire car beaucoup de classes imbriquées les unes dans les autres

What I have done so far:
* study the Hugging Face code from S.D.1.4 in detail
* read the paper (-> there's an error in the schema but overall a good paper)
* try to run TokenCompose (-> I had a library problem)
* recode my own TokenCompose using my knowledge of the paper and Hugging Face modules. I figured this would ultimately be the best way to understand what was going on in TokenCompose and unet. I didn't quite understand how to get the activation map (in the attentions layers) because dimension [1,4096,320] (I understood that 4096 is for 64x64 because the latent space is 64x64 but I didn't understand the relationship between 320 and the dimensions of the text embedding.

What I learned about TokenCompose:
* well-written paper except for a small error on figure 3 (the arrows are not connected correctly).
* I've realised that the real difficulty they've had to face is getting the attentions back (because I haven't managed that yet)... but I've understood that theoretically with a `hook` it should be possible.

General lessons:

* I learnt how to handle Hugging Face (the platform)
* I realised that Hugging Face is designed for models to be used, but not necessarily fine-tuned. For example, SD14 has no forward function, only a __call__ function.
* You can easily access the template code on Hugging Face, but it's difficult to read because there are so many classes nested inside each other.